In [72]:
import pandas as pd
import numpy as np
import joblib
import xgboost as xgb
from pathlib import Path

In [73]:
ARTIFACTS_DIR = Path('../../output/models/alt_models')
PROCESS_DIR = Path('../../output/data')

In [74]:
model = joblib.load(ARTIFACTS_DIR / 'xgb_alt_data_model.pkl')
encoders = joblib.load(PROCESS_DIR / 'label_encoders.pkl')
model_features = joblib.load(ARTIFACTS_DIR / 'model_features_list.pkl')
print("Hệ thống đã sẵn sàng!")

Hệ thống đã sẵn sàng!


In [ ]:
print("\n=== FORM ĐĂNG KÝ VAY TÍN CHẤP (Dành cho KH chưa có lịch sử tín dụng) ===")

# --- PHẦN A: THÔNG TIN CÁ NHÂN ---
input_age = 18                          # "Bạn bao nhiêu tuổi?"
input_region = "HN"                     # "Bạn đang sống ở đâu? (HN, HCM, OTHER)"
input_education = "high_school"          # "Trình độ học vấn cao nhất? (high_school, university, master...)"
input_school_tier = "tier_3"            # "Xếp hạng trường ĐH của bạn? (tier_1: Top đầu, tier_2: Khá, tier_3: TB)"
# --- PHẦN B: SỬ DỤNG ĐIỆN THOẠI (Telco) ---
input_sim_years = 1.0                   # "Bạn đã dùng số điện thoại này bao nhiêu năm?"
input_monthly_bill = 100000          # "Cước điện thoại trung bình 1 tháng (VND)?"
input_data_usage = "medium"               # "Mức độ dùng 4G của bạn? (high: >5GB, medium: 1-5GB, low: <1GB)"
input_recharge_habit = "regular"        # "Bạn nạp thẻ có đều đặn không? (regular: Đều, irregular: Thất thường)"

# --- PHẦN C: VÍ ĐIỆN TỬ (E-Wallet) ---
input_wallet_usage = True               # "Bạn có sử dụng ví điện tử không?"
input_wallet_balance = 500000         # "Số dư trung bình trong ví khoảng bao nhiêu (VND)?"
input_txn_frequency = 5                # "Số lượng giao dịch trung bình mỗi tháng?"
input_payment_failure = "never"         # "Bạn có hay bị lỗi thanh toán do thiếu tiền không? (never, sometimes, often)"
input_big_spending = "rarely"           # "Bạn có thường xuyên chi tiêu lớn (trên 5 triệu) không? (never, rarely, sometimes, often)"


=== FORM ĐĂNG KÝ VAY TÍN CHẤP (Dành cho KH chưa có lịch sử tín dụng) ===


In [76]:
def map_user_input_to_raw_data():
    # 1. Khởi tạo dict chứa dữ liệu thô
    raw = {}
    
    # 2. Map thông tin cá nhân
    raw['user_age'] = input_age
    raw['user_region'] = input_region
    raw['edu_highest_level'] = input_education
    raw['edu_institution_tier'] = input_school_tier
    raw['has_academic_data'] = 1 if input_education != 'none' else 0
    
    # 3. Map Telco (Quy đổi năm -> ngày, Habit -> Count)
    raw['has_telco_data'] = 1 # Giả định ai cũng có sim
    raw['telco_account_age_days'] = input_sim_years * 365
    raw['telco_avg_revenue_mean'] = input_monthly_bill
    
    # Logic quy đổi hành vi sang số liệu giả định (Proxy Logic)
    if input_recharge_habit == 'regular':
        raw['telco_recharge_count_mean'] = 4.0 # 1 tuần/lần
    else:
        raw['telco_recharge_count_mean'] = 1.0 # 1 tháng/lần
        
    if input_data_usage == 'high':
        raw['telco_mobile_data_mb_mean'] = 10000
    elif input_data_usage == 'medium':
        raw['telco_mobile_data_mb_mean'] = 3000
    else:
        raw['telco_mobile_data_mb_mean'] = 500
        
    # 4. Map Wallet
    if input_wallet_usage:
        raw['has_ewallet_data'] = 1
        raw['wallet_avg_amount'] = input_wallet_balance
        raw['wallet_txn_count'] = input_txn_frequency
        
        # Map failure rate (quan trọng cho FICO)
        fail_map = {'never': 0.0, 'sometimes': 0.1, 'often': 0.5}
        raw['wallet_failure_rate'] = fail_map.get(input_payment_failure, 0.0)
        
        # Map large txn ratio
        large_txn_map = {'often': 0.3, 'rarely': 0.05}
        raw['wallet_large_txn_ratio'] = large_txn_map.get(input_big_spending, 0.0)
    else:
        raw['has_ewallet_data'] = 0
        # Các trường khác tự động là 0 hoặc NaN ở bước sau
    
    return raw

In [77]:
def process_and_score(raw_data):
    df = pd.DataFrame([raw_data])
    
    # A. Impute Missing (Giống Notebook 01)
    numeric_cols = [
        'telco_account_age_days', 'telco_avg_revenue_mean', 
        'telco_recharge_count_mean', 'telco_mobile_data_mb_mean',
        'wallet_txn_count', 'wallet_avg_amount', 
        'wallet_large_txn_ratio', 'wallet_failure_rate'
    ]
    for col in numeric_cols:
        if col not in df.columns: df[col] = 0
        df[col] = df[col].fillna(0)
            
    # B. Tạo biến FICO (QUAN TRỌNG NHẤT)
    # 1. Payment Reliability: Nạp thẻ đều (+) trừ đi Lỗi thanh toán (-)
    df['fico_payment_reliability'] = df['telco_recharge_count_mean'] - (df['wallet_failure_rate'] * 100)
    
    # 2. Capacity: Tổng tiền cước + Số dư ví
    df['fico_financial_capacity'] = df['telco_avg_revenue_mean'] + df['wallet_avg_amount']
    
    # 3. History: Tuổi sim + (Số giao dịch ví * 10)
    df['fico_history_tenure'] = df['telco_account_age_days'] + (df['wallet_txn_count'] * 10)
    
    # 4. New Credit: Tỷ lệ giao dịch lớn
    df['fico_new_credit_intensity'] = df['wallet_large_txn_ratio']
    
    # 5. Mix Score: Telco(1) + Wallet(1) + Degree(1)
    has_degree = 1 if str(raw_data.get('edu_highest_level')) in ['university', 'master', 'phd'] else 0
    df['fico_credit_mix_score'] = df['has_telco_data'] + df['has_ewallet_data'] + has_degree

    # C. Encoding (Mã hóa biến chữ)
    # Map Tier
    tier_map = {'Unknown': 0, 'tier_3': 1, 'tier_2': 2, 'tier_1': 3}
    tier_val = str(raw_data.get('edu_institution_tier', 'Unknown'))
    df['edu_institution_tier_encoded'] = tier_map.get(tier_val, 0)
    
    # Label Encode Region (Dùng encoder đã train)
    region_val = str(raw_data.get('user_region', 'Unknown'))
    if 'user_region' in encoders:
        le = encoders['user_region']
        # Handle new labels safely
        if region_val in le.classes_:
            df['user_region_encoded'] = le.transform([region_val])
        else:
            df['user_region_encoded'] = 0 
    else:
        df['user_region_encoded'] = 0

    # D. Sắp xếp cột đúng chuẩn Model
    # Tạo các cột còn thiếu với giá trị 0
    for col in model_features:
        if col not in df.columns:
            df[col] = 0
    
    return df[model_features]

In [78]:
# Bước 1: Thu thập dữ liệu từ Form
raw_input = map_user_input_to_raw_data()
print("\n[System] Đã nhận dữ liệu thô từ người dùng...")

# Bước 2: Xử lý features
final_features = process_and_score(raw_input)
print("[System] Đã tính toán các chỉ số FICO (Reliability, Capacity, Mix)...")

# Bước 3: Dự báo
pd_prob = model.predict_proba(final_features)[0, 1]

# Bước 4: Tính điểm (Scaling)
def get_credit_score(pd):
    pdo = 20; target = 600; odds = 50
    factor = pdo / np.log(2)
    offset = target - (factor * np.log(odds))
    score = offset + (factor * np.log((1-pd)/pd)) # 1=Bad -> Odds=(1-p)/p
    return int(np.clip(score, 300, 850))

score = get_credit_score(pd_prob)

# --- 6. HIỂN THỊ KẾT QUẢ CHO NGƯỜI DÙNG ---
print("\n" + "="*40)
print(f"   KẾT QUẢ CHẤM ĐIỂM TÍN DỤNG (Alternative)")
print("="*40)
print(f"ĐIỂM SỐ CỦA BẠN: {score}/850")
print(f"Xác suất rủi ro: {pd_prob:.2%}")
print("-" * 40)

if score >= 700:
    print("XUẤT SẮC: Bạn đủ điều kiện vay lãi suất ưu đãi nhất (Tier 1).")
elif score >= 600:
    print("KHÁ: Bạn đủ điều kiện vay tiêu chuẩn.")
else:
    print("THẤP: Hồ sơ của bạn chưa đạt chuẩn. Hãy cải thiện:")
    if raw_input['wallet_failure_rate'] > 0:
        print("   - Giảm tỷ lệ lỗi thanh toán ví (đảm bảo số dư).")
    if raw_input['telco_account_age_days'] < 365:
        print("   - Duy trì số điện thoại lâu hơn.")
print("="*40)


[System] Đã nhận dữ liệu thô từ người dùng...
[System] Đã tính toán các chỉ số FICO (Reliability, Capacity, Mix)...

   KẾT QUẢ CHẤM ĐIỂM TÍN DỤNG (Alternative)
ĐIỂM SỐ CỦA BẠN: 793/850
Xác suất rủi ro: 0.00%
----------------------------------------
XUẤT SẮC: Bạn đủ điều kiện vay lãi suất ưu đãi nhất (Tier 1).
